In [ ]:
# EDS 6397 - Natural Language Processing
# Assignment #5 - Sentence Pair Classification - Task 1: Paraphrase Detection

# Install necessary packages (if not already installed)
!pip install transformers datasets evaluate torch scikit-learn matplotlib

# Import necessary libraries
import torch
import numpy as np
import evaluate
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import (
    RobertaTokenizer, 
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from torch.nn.utils import parameters_to_vector

# Set seed for reproducibility
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the MRPC dataset
dataset = load_dataset("glue", "mrpc")
print(dataset)

# Examine a sample
print(dataset["train"][0])

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare datasets
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

# Define metrics computation function
metric = evaluate.load("glue", "mrpc")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# PART 1: FINE-TUNING WITH FROZEN BASE MODEL
# ==========================================

# Load pre-trained model
model_frozen = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", 
    num_labels=2
)

# Freeze all parameters except the classification head
for param in model_frozen.roberta.parameters():
    param.requires_grad = False

# Count trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

frozen_params = count_parameters(model_frozen)
print(f"Number of trainable parameters (frozen base model): {frozen_params}")

# Define training arguments
training_args_frozen = TrainingArguments(
    output_dir="./results_mrpc_frozen",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,  # Higher learning rate for fewer trainable parameters
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,  # Min 10, max 20 epochs as per instructions
    weight_decay=0.01,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
    report_to="none",  # Disable wandb
    run_name="mrpc_frozen"  # Explicitly set run name
)

# Set up early stopping
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

# Initialize trainer
trainer_frozen = Trainer(
    model=model_frozen,
    args=training_args_frozen,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

# Train the model
print("Training frozen model...")
trainer_frozen.train()

# Evaluate on validation set
eval_results_frozen = trainer_frozen.evaluate()
print(f"Validation results (frozen model): {eval_results_frozen}")

# Evaluate on test set
test_results_frozen = trainer_frozen.predict(test_dataset)
print(f"Test results (frozen model):")
print(f"Accuracy: {test_results_frozen.metrics['test_accuracy']}")

# Get detailed metrics for each class
test_preds = np.argmax(test_results_frozen.predictions, axis=1)
test_labels = test_results_frozen.label_ids
class_report = classification_report(test_labels, test_preds, target_names=["Not Paraphrase", "Paraphrase"], output_dict=True)
print(classification_report(test_labels, test_preds, target_names=["Not Paraphrase", "Paraphrase"]))

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
loss_values = [x.get('loss', 0) for x in trainer_frozen.state.log_history if 'loss' in x]
plt.plot(loss_values)
plt.title("Training Loss (Frozen)")
plt.xlabel("Step")
plt.ylabel("Loss")

plt.subplot(1, 2, 2)
eval_results = [x.get('eval_accuracy', 0) for x in trainer_frozen.state.log_history if 'eval_accuracy' in x]
plt.plot(eval_results)
plt.title("Validation Accuracy (Frozen)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig("mrpc_frozen_training.png")
plt.show()

# PART 2: FINE-TUNING WITH UNFROZEN MODEL
# =======================================

# Load pre-trained model again (all weights trainable)
model_unfrozen = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", 
    num_labels=2
)

# Count trainable parameters
unfrozen_params = count_parameters(model_unfrozen)
print(f"Number of trainable parameters (unfrozen model): {unfrozen_params}")

# Define training arguments (smaller learning rate for all parameters)
training_args_unfrozen = TrainingArguments(
    output_dir="./results_mrpc_unfrozen",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,  # Lower learning rate when fine-tuning all parameters
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,  # Min 3, max 5 epochs as per instructions
    weight_decay=0.01,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
    report_to="none",  # Disable wandb
    run_name="mrpc_unfrozen"  # Explicitly set run name
)

# Initialize trainer
trainer_unfrozen = Trainer(
    model=model_unfrozen,
    args=training_args_unfrozen,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
print("Training unfrozen model...")
trainer_unfrozen.train()

# Evaluate on validation set
eval_results_unfrozen = trainer_unfrozen.evaluate()
print(f"Validation results (unfrozen model): {eval_results_unfrozen}")

# Evaluate on test set
test_results_unfrozen = trainer_unfrozen.predict(test_dataset)
print(f"Test results (unfrozen model):")
print(f"Accuracy: {test_results_unfrozen.metrics['test_accuracy']}")

# Get detailed metrics for each class
test_preds_unfrozen = np.argmax(test_results_unfrozen.predictions, axis=1)
test_labels_unfrozen = test_results_unfrozen.label_ids
class_report_unfrozen = classification_report(test_labels_unfrozen, test_preds_unfrozen, target_names=["Not Paraphrase", "Paraphrase"], output_dict=True)
print(classification_report(test_labels_unfrozen, test_preds_unfrozen, target_names=["Not Paraphrase", "Paraphrase"]))

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
loss_values = [x.get('loss', 0) for x in trainer_unfrozen.state.log_history if 'loss' in x]
plt.plot(loss_values)
plt.title("Training Loss (Unfrozen)")
plt.xlabel("Step")
plt.ylabel("Loss")

plt.subplot(1, 2, 2)
eval_results = [x.get('eval_accuracy', 0) for x in trainer_unfrozen.state.log_history if 'eval_accuracy' in x]
plt.plot(eval_results)
plt.title("Validation Accuracy (Unfrozen)")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig("mrpc_unfrozen_training.png")
plt.show()

# COMPARE RESULTS
frozen_metrics = {
    "accuracy": test_results_frozen.metrics['test_accuracy'],
    "not_paraphrase_precision": class_report["Not Paraphrase"]["precision"],
    "not_paraphrase_recall": class_report["Not Paraphrase"]["recall"],
    "not_paraphrase_f1": class_report["Not Paraphrase"]["f1-score"],
    "paraphrase_precision": class_report["Paraphrase"]["precision"],
    "paraphrase_recall": class_report["Paraphrase"]["recall"],
    "paraphrase_f1": class_report["Paraphrase"]["f1-score"],
}

unfrozen_metrics = {
    "accuracy": test_results_unfrozen.metrics['test_accuracy'],
    "not_paraphrase_precision": class_report_unfrozen["Not Paraphrase"]["precision"],
    "not_paraphrase_recall": class_report_unfrozen["Not Paraphrase"]["recall"],
    "not_paraphrase_f1": class_report_unfrozen["Not Paraphrase"]["f1-score"],
    "paraphrase_precision": class_report_unfrozen["Paraphrase"]["precision"],
    "paraphrase_recall": class_report_unfrozen["Paraphrase"]["recall"],
    "paraphrase_f1": class_report_unfrozen["Paraphrase"]["f1-score"],
}

# Print comparison
print("\nModel Comparison:")
print(f"{'Metric':<25} {'Frozen Base':<15} {'Unfrozen':<15}")
print("-" * 55)
for metric in frozen_metrics:
    print(f"{metric:<25} {frozen_metrics[metric]:<15.4f} {unfrozen_metrics[metric]:<15.4f}")

# Save models (optional)
trainer_frozen.save_model("./final_mrpc_frozen")
trainer_unfrozen.save_model("./final_mrpc_unfrozen")